## Building a RAG to predit risk score of a transaction 

### Initiating model using groq inference server

In [1]:
import getpass
import os

if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")

os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "Anish@hack1"


from langchain.chat_models import init_chat_model

model = init_chat_model("qwen-2.5-coder-32b", model_provider="groq")

### Simple prompt with SystemMessage and HumanMessage

In [3]:
from langchain_core.messages import HumanMessage, SystemMessage

transaction = input('Enter transaction: ')

messages = [
    SystemMessage("You are an assitant that helps data analysts in a financial institution to risk score a transaction among entities/corporations. It can also include individuals. Given an input transaction, you need to predict the risk score (0 to 1) of the transaction, confidence score (0 to 1) and a reason for the risk score."),
    HumanMessage(transaction),
]

model.invoke(messages)

AIMessage(content='To predict the risk score for the transaction of transferring 10 rupees from Wells Fargo to Microsoft, we need to consider several factors, such as the amount, the entities involved, the purpose, and the context. Here’s a breakdown:\n\n### Factors to Consider:\n1. **Amount:** The transaction amount is very small (10 rupees), which typically does not pose a significant risk in terms of financial fraud.\n2. **Entities Involved:** Wells Fargo is a reputable financial institution, and Microsoft is a well-known technology company with a strong financial standing.\n3. **Purpose:** Without additional context, the purpose of this transaction is unclear. If this transaction is part of a legitimate business relationship, it would likely have a lower risk.\n4. **Context:** There is no additional context provided, such as frequency, historical transactions, or the relationship between the two entities.\n\n### Predicted Risk Score:\n- **Risk Score:** 0.02\n- **Confidence Score:**

### Creating a Chain with prompt (chatprompttemplate) and context (wikipedia retriever) and model 

#### Wikipedia Retriever 

In [2]:
from langchain_community.retrievers import WikipediaRetriever

# documentloader 
wiki_retriever = WikipediaRetriever()

In [4]:
docs = wiki_retriever.invoke("Adani Green Energy")
for doc in docs:
    print(doc)
    print("===")

# print(docs[2].page_content[:400])

page_content='Adani Green Energy Limited (AGEL) is an Indian renewable energy company, headquartered in Ahmedabad, India. It is majority-owned by Indian conglomerate Adani Group and minority-owned by TotalEnergies. The company operates Kamuthi Solar Power Project, one of the largest solar photovoltaic plants in the world.


== History ==
The company was incorporated on 23 January 2016, as Adani Green Energy Limited under the Companies Act 2013.
During the initial days of existence, AGEL and Inox Wind together established a 20 MW capacity wind power project in Lahori, Madhya Pradesh. Also, AGEL bought Inox Wind's 50 MW wind power project at Dayapar village in Kutch. The project was conceived by the latter when it won a Solar Energy Corporation of India's capacity bids for wind power projects connected to the National Grid.
In 2015–2016, Adani Renewable Energy Park Limited, a subsidiary of AGEL, signed a joint venture agreement with the Government of Rajasthan.
In 2017, the company took 

#### Chain 

In [5]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

prompt = ChatPromptTemplate.from_template(
    """
    You are an agent that helps data analysts in a financial institution by risk scoring a transaction 
    among entities/corporations. It can also include individuals. Given an input transaction, you need to output 
    the risk score (0 to 1) of the transaction, confidence score (0 to 1). Use the context if needed.
    Context: {context}
    Transaction: {transaction}
    """
)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

chain = (
    {"context": wiki_retriever | format_docs, "transaction": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)
# chain = (
#     {"context": retriever, "transaction": RunnablePassthrough()},
#     | prompt
#     | model
#     | StrOutputParser()
# )

In [6]:
# transaction_test = input()
chain.invoke(
    "Transaction ID: TXN20250322113045, Date: 2025-03-22T11:30:45Z,  Payer: Tesla, Inc.  Payer Account Number: TESLA987654321,  Payer Bank: JPMorgan Chase Bank, USA, Payee: Adani Green Energy Ltd.  ,Payee Account Number: ADANIGREEN123456  ,Payee Bank: State Bank of India, India  ,Amount: 500,000,000 USD  ,Payment Method: Wire Transfer  ,Purpose: Investment in Renewable Energy Collaboration  ,Status: Completed  "
)

'To evaluate the risk score and confidence score for the given transaction, we need to consider several factors such as the entities involved, the amount of the transaction, the purpose of the transaction, and any potential red flags.\n\n### Key Points:\n1. **Entities Involved**:\n   - **Payer**: Tesla, Inc. - A well-known and reputable company in the technology and automotive sectors.\n   - **Payee**: Adani Green Energy Ltd. - A prominent Indian company in renewable energy and infrastructure.\n\n2. **Amount**: \n   - $500,000,000 USD - This is a very large transaction, which could raise some red flags due to the scale and the need for thorough due diligence.\n\n3. **Purpose**:\n   - Investment in Renewable Energy Collaboration - This aligns with the business profiles of both Tesla and Adani Green Energy, suggesting a legitimate purpose.\n\n4. **Payment Method**:\n   - Wire Transfer - A standard method for large international transactions.\n\n5. **Geographical Considerations**:\n   - T

In [6]:
from langchain_neo4j import Neo4jGraph

graph = Neo4jGraph()


In [7]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Entity {countries: STRING, ibcRUC: STRING, valid_until: STRING, country_codes: STRING, service_provider: STRING, address: STRING, inactivation_date: STRING, struck_off_date: STRING, status: STRING, jurisdiction_description: STRING, incorporation_date: STRING, original_name: STRING, jurisdiction: STRING, name: STRING, internal_id: STRING, lastEditTimestamp: STRING, node_id: INTEGER, sourceID: STRING, former_name: STRING, company_type: STRING, tax_stat_description: STRING, note: STRING, dorm_date: STRING, type: STRING, closed_date: STRING, company_number: STRING, comments: STRING, entity_number: STRING}
Intermediary {countries: STRING, lastEditTimestamp: STRING, address: STRING, valid_until: STRING, country_codes: STRING, name: STRING, status: STRING, node_id: INTEGER, sourceID: STRING, internal_id: STRING, note: STRING, registered_office: STRING}
Officer {valid_until: STRING, sourceID: STRING, name: STRING, icij_id: STRING, node_id: INTEGER, lastEditTimestamp: STRING, c

In [8]:
enhanced_graph = Neo4jGraph(enhanced_schema=True)
print(enhanced_graph.schema)

Node properties:
- **Entity**
  - `countries`: STRING Example: "Hong Kong"
  - `ibcRUC`: STRING Example: "25221"
  - `valid_until`: STRING Example: "The Panama Papers data is current through 2015"
  - `country_codes`: STRING Example: "HKG"
  - `service_provider`: STRING Available options: ['Appleby', 'Portcullis Trustnet', 'Mossack Fonseca', 'Commonwealth Trust Limited']
  - `address`: STRING Example: "ORION HOUSE SERVICES (HK) LIMITED ROOM 1401; 14/F."
  - `inactivation_date`: STRING Example: "18-FEB-2013"
  - `struck_off_date`: STRING Example: "15-FEB-2013"
  - `status`: STRING Example: "Defaulted"
  - `jurisdiction_description`: STRING Example: "Samoa"
  - `incorporation_date`: STRING Example: "23-MAR-2006"
  - `original_name`: STRING Example: "TIANSHENG INDUSTRY AND TRADING CO., LTD."
  - `jurisdiction`: STRING Example: "SAM"
  - `name`: STRING Example: "TIANSHENG INDUSTRY AND TRADING CO., LTD."
  - `internal_id`: STRING Example: "1001256"
  - `lastEditTimestamp`: STRING Example: "

In [ ]:
## Note: Sometimes this may return an error. It depends on the how well the llm is able to generate the cypher query. 
## Few-shot prompting is one way to improve the performance of the llm.
from langchain_neo4j import GraphCypherQAChain

chain = GraphCypherQAChain.from_llm(
    graph=enhanced_graph, llm=model, verbose=True, allow_dangerous_requests=True, validate_cypher=True
)
response = chain.invoke({"query": "Name the officers that are linked with French entities"})
print(response) # response becomes context 


## Input: Transaction -> Process the input to extract entities and generate a prompt to generate a query -> Query the graph database to get the required information and store it -> Pass this along with Wikipedia retriever to generate the risk_score and confidence score 
## Should we use separate chains? Is it possible to use one chain to do all of this?

## Output: Risk Score, Confidence Score, Reason for the risk score

## catching error, timeout ,



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (o:Officer)-[:officer_of]->(e:Entity {countries: "France"})
RETURN o.name
Full Context:
[{'o.name': 'Emilie Makaci'}, {'o.name': 'Michael Makaci'}, {'o.name': 'THE BEARER'}, {'o.name': 'THE BEARER'}, {'o.name': 'MRS. KITTY GABLE'}, {'o.name': 'Kitty Jeanne Gable'}, {'o.name': 'Mr. Dragan Milosavljevi'}, {'o.name': 'THE BEARER'}, {'o.name': 'THE BEARER'}, {'o.name': 'THE BEARER'}]

> Finished chain.
{'query': 'Name the officers that are linked with French entities', 'result': 'Emilie Makaci, Michael Makaci, MRS. KITTY GABLE, and Kitty Jeanne Gable are the officers linked with French entities.'}


In [28]:
import os

api_key = input("Enter your OpenSanctions API Key: ")
os.environ["OPENSANCTIONS_API_KEY"] = api_key

In [29]:
from pprint import pprint
import requests
import os

OS_API_KEY = os.getenv("OPENSANCTIONS_API_KEY")
if not OS_API_KEY:
    raise ValueError("The OS_API_KEY environment variable is not set")

def fetch_sanctions_data(person_name, company_name):
    headers = {"Authorization": OS_API_KEY}

    query = {
        "queries": {
            "query-A": {"schema": "Person", "properties": {"name": [person_name]}},
            "query-B": {"schema": "Company", "properties": {"name": [company_name]}},
        }
    }

    response = requests.post(
        "https://api.opensanctions.org/match/default", headers=headers, json=query
    )
    response.raise_for_status()
    response_json = response.json()

    print("\nFull API Response:")
    pprint(response_json, sort_dicts=False)

    # Define known risky datasets
    risky_datasets = {
        "us_ofac_sdn", "eu_travel_bans", "gb_hmt_sanctions", "ua_nsdc_sanctions",
        "eu_fsf", "lt_fiu_freezes", "ca_dfatd_sema_sanctions", "jp_mof_sanctions",
        "ch_seco_sanctions", "ru_acf_bribetakers", "be_fod_sanctions", "au_dfat_sanctions"
    }
    
    for query_id, query_response in response_json["responses"].items():
        print(f"\nResults for query {query_id}:")
        results = []
        
        for result in query_response["results"]:
            entity_info = {
                "id": result["id"],
                "name": result["properties"].get("name", []),
                "match": result["match"],
                "topics": result["properties"].get("topics", []),
                "datasets": result.get("datasets", []),
            }
            results.append(entity_info)

        pprint(results, sort_dicts=False)

        # **Improved Risk Detection**
        risky_entities = []
        for entity in results:
            entity_topics = set(entity.get("topics", []))
            entity_datasets = set(entity.get("datasets", []))

            if entity_topics.intersection({"sanction", "pep", "crime", "corruption", "terrorism"}) or \
               entity_datasets.intersection(risky_datasets):
                risky_entities.append(entity)

        if risky_entities:
            print("\n🚨 Risky Entities Detected:")
            pprint(risky_entities, sort_dicts=False)
        else:
            print("\n✅ No risky entities found.")

# Example Usage:
fetch_sanctions_data("Vladimir Putin", "")



# "name": result["properties"].get("name", []),
                    # "match": result["match"],
                    # "topics": list(entity_topics),  
                    # "datasets": list(entity_datasets),  


Full API Response:
{'responses': {'query-A': {'status': 200,
                           'results': [{'id': 'Q7747',
                                        'caption': 'Vladimir Putin',
                                        'schema': 'Person',
                                        'properties': {'birthDate': ['1952-10-07'],
                                                       'position': ['Secretary '
                                                                    'of the '
                                                                    'Security '
                                                                    'Council '
                                                                    'of Russia '
                                                                    '(1999-1999)',
                                                                    'Supreme '
                                                                    'Commander-in-Chief '
                   

In [ ]:
from pprint import pprint
import requests
import os

OS_API_KEY = os.getenv("OPENSANCTIONS_API_KEY")
if not OS_API_KEY:
    raise ValueError("The OS_API_KEY environment variable is not set")

def fetch_sanctions_data(person_name, company_name):
    headers = {"Authorization": OS_API_KEY}

    query = {
        "queries": {
            "query-A": {"schema": "Person", "properties": {"name": [person_name]}},
            "query-B": {"schema": "Company", "properties": {"name": [company_name]}},
        }
    }

    response = requests.post(
        "https://api.opensanctions.org/match/default", headers=headers, json=query
    )
    response.raise_for_status()
    response_json = response.json()

    print("\nFull API Response:")
    pprint(response_json, sort_dicts=False)

    # Define known risky datasets and topics
    risky_datasets = {
    "eu_travel_bans", "gb_hmt_sanctions", "ua_nsdc_sanctions", "jp_mof_sanctions",
    "ca_dfatd_sema_sanctions", "ch_seco_sanctions", "lt_fiu_freezes", "au_dfat_sanctions",
    "us_ofac_sdn", "ru_acf_bribetakers", "be_fod_sanctions", "eu_fsf",
    "in_nse_debarred"  # ✅ Added NSE debarred list
}


    risky_topics = {"wanted", "sanction", "poi", "corruption", "crime", "role.pep", "terrorism", "reg.warn"}


    for query_id, query_response in response_json["responses"].items():
        print(f"\nResults for query {query_id}:")
        results = []
        
        for result in query_response["results"]:
            entity_topics = set(result["properties"].get("topics", []))  # Fix extraction
            entity_datasets = set(result.get("datasets", []))  # Fix extraction

            print(f"\n🔎 Checking Entity: {result['id']}")
            print(f"📌 Topics: {entity_topics}")
            print(f"📌 Datasets: {entity_datasets}")

            entity_info = {
                "id": result["id"],
                "name": result["properties"].get("name", []),
                "match": result["match"],
                "topics": list(entity_topics),  
                "datasets": list(entity_datasets),  
            }
            results.append(entity_info)

        pprint(results, sort_dicts=False)

        # **Improved Risk Detection**
        risky_entities = []
        for entity in results:
            entity_topics = set(entity["topics"])
            entity_datasets = set(entity["datasets"])

            print(f"\n🧐 Checking Risk for {entity['id']}:")
            print(f"🟡 Entity Topics: {entity_topics}")
            print(f"🟡 Risky Topics: {risky_topics}")
            print(f"🔹 Matching Topics: {entity_topics.intersection(risky_topics)}")

            print(f"🟢 Entity Datasets: {entity_datasets}")
            print(f"🟢 Risky Datasets: {risky_datasets}")
            print(f"🔸 Matching Datasets: {entity_datasets.intersection(risky_datasets)}")

            if entity_topics.intersection(risky_topics) or entity_datasets.intersection(risky_datasets):
                risky_entities.append(entity)

        if risky_entities:
            print("\n🚨 Risky Entities Detected:")
            pprint(risky_entities, sort_dicts=False)
        else:
            print("\n✅ No risky entities found.")

# Example Usage:
fetch_sanctions_data("Adani",  "Adani Green Energy Ltd.")



Full API Response:
{'responses': {'query-A': {'status': 200,
                           'results': [{'id': 'in-nse-deb-f55b2ae39dc2ff4ce90ba179a4a6992d83b841b1',
                                        'caption': 'RAJESHBHAI SHANTILAL ADANI',
                                        'schema': 'LegalEntity',
                                        'properties': {'taxNumber': ['ABKPA0962A'],
                                                       'jurisdiction': ['in'],
                                                       'topics': ['reg.warn'],
                                                       'name': ['RAJESHBHAI '
                                                                'SHANTILAL '
                                                                'ADANI']},
                                        'datasets': ['in_nse_debarred'],
                                        'referents': [],
                                        'target': True,
                                 

In [ ]:
#added changes to the branch

In [82]:
import requests
from langchain.schema.document import Document
from langchain.schema.retriever import BaseRetriever

class OpenSanctionsRetriever(BaseRetriever):

    def _get_relevant_documents(self, company_name, person_name, api_key):
        """
        Queries OpenSanctions API and returns relevant documents.
        """
        headers = {"Authorization": api_key}
        # params = {"q": query}

        query = {
            "queries": {
                "query-A": {"schema": "Person", "properties": {"name": [person_name]}},
                "query-B": {"schema": "Company", "properties": {"name": [company_name]}},
            }
        }
        response = requests.post(
            "https://api.opensanctions.org/match/default", headers=headers, json=query
        )
        # if response.status_code != 200:
        #     return []
        
        response.raise_for_status()
        response_json = response.json()

        # print("\nFull API Response:")
        # pprint(response_json, sort_dicts=False)

        # if not response_json.get("results"):
        #     print("empty list lool")
        #     return []

        documents = []
        # print("outside first for")
        for query_id, query_response in response_json["responses"].items():
            # print(f"\nResults for query {query_id}:")
            # results = []
            
            for result in query_response["results"]:
                # print("in for result")
                entity_topics = set(result["properties"].get("topics", []))  # Fix extraction
                entity_datasets = set(result.get("datasets", []))  # Fix extraction

                # print(f"\n🔎 Checking Entity: {result['id']}")
                # print(f"📌 Topics: {entity_topics}")
                # print(f"📌 Datasets: {entity_datasets}")
                
                name_to_store_page_content=result["properties"].get("name")
                # print("hehe", name_to_store_page_content[0])
                entity_info = {
                    "id": result["id"],
                    "name": result["properties"].get("name", []),
                    "match": result["match"],
                    "topics": list(entity_topics),  
                    "datasets": list(entity_datasets),  
                }
                doc = Document(page_content=f"Sanctions data for {name_to_store_page_content}", metadata=entity_info)
                # print("doc = ", doc)
                documents.append(doc)
                # results.append(entity_info)
        return documents


In [83]:
OpenSanctions_retriever = OpenSanctionsRetriever()
sanction_docs = OpenSanctions_retriever._get_relevant_documents("" ,"Adani", api_key="3b9678eb2e0dff14c268b43f7acf4798")
print("sanction_docs = ", sanction_docs)
# print(sanction_docs)

sanction_docs =  [Document(metadata={'id': 'in-nse-deb-f55b2ae39dc2ff4ce90ba179a4a6992d83b841b1', 'name': ['RAJESHBHAI SHANTILAL ADANI'], 'match': True, 'topics': ['reg.warn'], 'datasets': ['in_nse_debarred']}, page_content="Sanctions data for ['RAJESHBHAI SHANTILAL ADANI']"), Document(metadata={'id': 'in-nse-deb-1bf180865bd3417cc52f23cf9c2bb553a6a5821b', 'name': ['Amit Kumar Adani'], 'match': True, 'topics': ['reg.warn'], 'datasets': ['in_nse_debarred']}, page_content="Sanctions data for ['Amit Kumar Adani']"), Document(metadata={'id': 'in-nse-deb-8e319548cd3c601ba88309e65694d7031535d550', 'name': ['ADANI ENTERPRISES LIMITED'], 'match': True, 'topics': ['reg.warn'], 'datasets': ['in_nse_debarred']}, page_content="Sanctions data for ['ADANI ENTERPRISES LIMITED']"), Document(metadata={'id': 'in-nse-deb-6f2973d1ce5202ad2a73386c8c28cf2993312edf', 'name': ['GAUTAMBHAI SHANTILAL ADANI'], 'match': True, 'topics': ['reg.warn'], 'datasets': ['in_nse_debarred']}, page_content="Sanctions data fo

In [ ]:
for doc in sanction_docs:
    print(doc)
    print("===")

page_content='Sanctions data for RAJESHBHAI SHANTILAL ADANI' metadata={'id': 'in-nse-deb-f55b2ae39dc2ff4ce90ba179a4a6992d83b841b1', 'name': ['RAJESHBHAI SHANTILAL ADANI'], 'match': True, 'topics': ['reg.warn'], 'datasets': ['in_nse_debarred']}
===
page_content='Sanctions data for Amit Kumar Adani' metadata={'id': 'in-nse-deb-1bf180865bd3417cc52f23cf9c2bb553a6a5821b', 'name': ['Amit Kumar Adani'], 'match': True, 'topics': ['reg.warn'], 'datasets': ['in_nse_debarred']}
===
page_content='Sanctions data for ADANI ENTERPRISES LIMITED' metadata={'id': 'in-nse-deb-8e319548cd3c601ba88309e65694d7031535d550', 'name': ['ADANI ENTERPRISES LIMITED'], 'match': True, 'topics': ['reg.warn'], 'datasets': ['in_nse_debarred']}
===
page_content='Sanctions data for GAUTAMBHAI SHANTILAL ADANI' metadata={'id': 'in-nse-deb-6f2973d1ce5202ad2a73386c8c28cf2993312edf', 'name': ['GAUTAMBHAI SHANTILAL ADANI'], 'match': True, 'topics': ['reg.warn'], 'datasets': ['in_nse_debarred']}
===
page_content='Sanctions data f

: 